In [1]:
import numpy as np 
import pandas as pd
from sklearn.linear_model import LogisticRegression as logreg
from sklearn.neural_network import MLPClassifier as nn
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.svm import SVC, LinearSVC
import matplotlib.pylab as plt
%matplotlib inline

pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 40)
print('hi')

hi


In [3]:
#pitches = pd.read_csv("pitch_test_data.csv")
#pitches['diff_pit_bat_h'] = pitches.apply(lambda row: (int(row['bat_hand_fl']) ^ int(row['pit_hand_fl'])), axis=1)
# xor hands of pitcher and batter, 1 if different handed pitcher and batter, 0 if same
#pitches.head(15)
single_encoding = pd.read_csv("pitch_data_single_encoding_filter.csv")
single_encoding['diff_pit_bat_h'] = single_encoding.apply(lambda row: (int(row['bat_hand_fl']) ^ int(row['pit_hand_fl'])), axis=1)



In [16]:
single_encoding.head()

,inning,bat_home_id,bat_hand_fl,pit_hand_fl,pa_ball_ct,pa_strike_ct,outs_ct,start_bases_cd,ab_number,x,y,end_speed,pitch_event,sz_top,sz_bot,pfx_x,pfx_z,px,pz,x0,y0,z0,vx0,vy0,vz0,ax,ay,az,break_y,break_angle,break_length,spin_dir,spin_rate,diff_pit_bat_h
0,1,0,0,0,0,0,0,0,1,3.00,104.72,93.6,3,86.20,3.61,1.82,-7.82,10.730,-0.111,3.206,-2,50.000,5.451,7.887,-137.019,-3.927,-14.894,29.614,-11.7,23.8,46.6,2.000,215.969,0
1,1,0,1,0,0,0,1,0,2,121.03,141.61,85.7,2,3.13,1.82,-13.07,10.51,-0.937,2.758,-2.332,50,5.192,8.407,-139.276,-4.440,-25.106,37.763,-11.921,23.7,60.1,5.6,231.106,3360.052,1
2,1,0,1,0,0,1,1,0,2,125.32,164.92,81.4,0,3.10,1.65,-4.68,12.11,-0.763,1.664,-2.044,50,5.242,4.822,-129.187,-6.863,-7.927,26.377,-11.593,23.8,28.3,3.5,201.063,2476.777,1
3,1,0,1,0,1,1,1,0,2,109.01,113.98,85.9,0,3.07,1.69,-11.58,10.03,-0.220,3.695,-2.142,50,5.344,9.275,-137.182,-2.063,-22.038,30.684,-13.001,23.8,56.0,5.1,228.976,3089.230,1
4,1,0,1,0,2,1,1,0,2,10.00,92.70,0.0,3,0.00,3.33,1.70,0.00,0.000,0.000,0.000,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,1


In [ ]:
#pitches = pd.read_csv("pitch_test_data.csv")
#pitches['diff_pit_bat_h'] = pitches.apply(lambda row: (int(row['bat_hand_fl']) ^ int(row['pit_hand_fl'])), axis=1)
# xor hands of pitcher and batter, 1 if different handed pitcher and batter, 0 if same


#corr_mat = pitches.corr(method='pearson')[['ball', 'called_strike', "swinging_strike", "ball_hit"]]

In [4]:
pitchCategories = ['x', 'y', 'end_speed', 'diff_pit_bat_h',
    #'ball', 'called_strike',
    #'swinging_strike', 
    #'ball_hit', 'sz_top', 'sz_bot', 
    'pfx_x', 'pfx_z', 'px', 'pz', 'x0', 'y0', 'z0', 'vx0', 'vy0', 'vz0', 'ax', 
    'ay', 'az', 'break_y', 'break_angle', 'break_length', 'spin_dir', 'spin_rate']
def separateData(pitchData):
    px = pitchData[pitchCategories].values
    
    py = pitchData[['pitch_event']].values[:, 0]
    #py = pitchData[['ball', 'swinging_strike', 'called_strike', 'ball_hit']].values
    return px, py


In [5]:


px, py = separateData(single_encoding)
#print(px, py)
#px_ = single_encoding[pitchCategories].values
#py_ = single_encoding[['pitch_event']].values[:, 0]

#print(py_)


#print(py_[:, 0])
#print(py_[0])

#corr_mat = p_.corr(method='pearson')[['ball', 'called_strike', "swinging_strike", "ball_hit"]]
#corr_mat = single_encoding.corr(method='pearson')[['pitch_event']]
#corr_mat
#corr_mat

In [6]:
#corr_mat.to_csv("../Project/pfx_parser/test_corr_mat.csv")
print(single_encoding[['pitch_event']].shape)
print(single_encoding[['pitch_event']].values[:,0].size)
print(single_encoding.shape)
#print(px.size)
print(len(py))
print(py.shape)
print(py[:10])


(901783, 1)
901783
(901783, 34)
901783
(901783,)
[2 0 0 0 2 2 0 3 1 2]


In [7]:
'''for el in p_:
    print(el)
    fig, ax = plt.subplots(figsize=(8,4))
    p_.hist(column=el, ax=ax);
'''

def getAccuracy(predict, actual):
    correct = 0
    if len(actual) != len(predict):
        raise ValueError('predict and actual arrays have different lengths')
    else:
        for i in range(len(actual) - 1):
            #print('i = ', i)
            #print('predict[i] =', predict[i])
            #print('actual[i] =', actual[i])
            if predict[i] == actual[i]:
                correct += 1
    
    return correct/len(predict)

def getCounts(arr):
    occurences = {}
    for item in arr:
        #print("item", item)
        if item in occurences:
            occurences[item] += 1
        else:
            occurences[item] = 1
    return occurences




In [8]:
halfPt = round(len(py)/2)

mdl2 = logreg()
mdl2.fit(px[:halfPt], py[:halfPt])
trainp = mdl2.predict(px[:halfPt])
testp = mdl2.predict(px[halfPt:])

traina = getAccuracy(trainp, py[:halfPt])
testa = getAccuracy(testp, py[halfPt:])

print('train accuracy', traina)
print('test accuracy', testa)


print('py_ array', getCounts(py))
print('train predictions', getCounts(trainp))
print('test predictions', getCounts(testp))

#print(mdl.score(px_[100000:], py_[100000:]))
                    

train accuracy 0.43644376036833654
test accuracy 0.43486341488297614
py_ array {2: 186737, 0: 403373, 3: 187438, 1: 124235}
train predictions {0: 443553, 3: 2901, 2: 4430, 1: 8}
test predictions {0: 443344, 3: 3027, 2: 4512, 1: 8}


In [9]:
halfPt = round(len(py)/2)

clf_ = knn(n_neighbors=50)
clf_.fit(px[:halfPt], py[:halfPt])
trainp = clf_.predict(px[:halfPt])
testp = clf_.predict(px[halfPt:])

traina = getAccuracy(trainp, py[:halfPt])
testa = getAccuracy(testp, py[halfPt:])

print('train accuracy', traina)
print('test accuracy', testa)


print('py_ array', getCounts(py))
print('train predictions', getCounts(trainp))
print('test predictions', getCounts(testp))

#print(mdl.score(px_[100000:], py_[100000:]))



train accuracy 0.6166665188116001
test accuracy 0.620047860791189
py_ array {2: 186737, 0: 403373, 3: 187438, 1: 124235}
train predictions {0: 240870, 3: 86236, 2: 105520, 1: 18266}
test predictions {0: 240351, 2: 105748, 3: 86245, 1: 18547}


In [7]:
#px_2, py_2 = separateData(mini_single_encoding)
halfPt = 50000 #round(len(py_2)/2)

clf = SVC()
clf.fit(px[:halfPt], py[:halfPt])
trainp = clf.predict(px[:halfPt])
testp = clf.predict(px[halfPt:])

traina = getAccuracy(trainp, py[:halfPt])
testa = getAccuracy(testp, py[halfPt:])

print('train accuracy', traina)
print('test accuracy', testa)

print('py_ array', getCounts(py))
print('train predictions', getCounts(trainp))
print('test predictions', getCounts(testp))


train accuracy 0.99902
test accuracy 0.41518842215511015
py_ array {3: 290824, 2: 141831, 0: 309838, 1: 94251}
train predictions {3: 17362, 2: 8754, 0: 19058, 1: 4826}
test predictions {0: 748944, 3: 22870, 2: 9573, 1: 5357}


In [10]:
halfPt = round(len(py)/2)
nnclf = nn()
nnclf.fit(px[:halfPt], py[:halfPt])
trainp = nnclf.predict(px[:halfPt])
testp = nnclf.predict(px[halfPt:])

traina = getAccuracy(trainp, py[:halfPt])
testa = getAccuracy(testp, py[halfPt:])

print('train accuracy', traina)
print('test accuracy', testa)

print('py_ array', getCounts(py))
print('train predictions', getCounts(trainp))
print('test predictions', getCounts(testp))


train accuracy 0.616100973182048
test accuracy 0.6164283607346344
py_ array {2: 186737, 0: 403373, 3: 187438, 1: 124235}
train predictions {3: 106000, 2: 106259, 0: 222316, 1: 16317}
test predictions {0: 222025, 2: 106486, 3: 105774, 1: 16606}


In [13]:
test_pitches = pd.read_csv("pitch_test_data_filtered.csv")
test_pitches['diff_pit_bat_h'] = test_pitches.apply(lambda row: (int(row['bat_hand_fl']) ^ int(row['pit_hand_fl'])), axis=1)
test_pitches.head()



,retro_game_id,inning,bat_home_id,pa_ball_ct,pa_strike_ct,outs_ct,start_bases_cd,ab_number,x,y,end_speed,sz_top,sz_bot,pfx_x,pfx_z,px,pz,x0,y0,z0,vx0,vy0,vz0,ax,ay,az,break_y,break_angle,break_length,spin_dir,spin_rate,pitch_event,pit_hand_fl,bat_hand_fl,diff_pit_bat_h
0,CHN201708010,1,0,0,0,0,0,1,173.42,208.82,85.1,3.501,1.664,6.847,8.763,-1.343,0.979,2.473,50,5.380,-12.461,-133.281,-8.591,12.493,24.807,-16.185,23.9,-30.1,4.5,141.999,2208.653,0,1,1,0
1,CHN201708010,1,0,1,0,0,0,1,151.63,191.63,84.1,3.521,1.673,6.465,8.980,-0.796,1.656,2.456,50,5.409,-10.758,-132.578,-6.811,11.617,25.811,-16.039,23.8,-29.0,4.5,144.249,2175.647,2,1,1,0
2,CHN201708010,1,0,1,1,0,0,1,164.24,206.29,83.9,3.316,1.397,0.792,8.910,-1.084,1.082,2.571,50,5.439,-9.762,-130.467,-8.142,1.394,22.161,-16.501,23.9,0.5,4.1,174.921,1749.724,1,1,1,0
3,CHN201708010,1,0,1,2,0,0,1,141.31,178.59,86.3,3.317,1.398,6.622,8.714,-0.487,2.152,2.499,50,5.380,-10.368,-136.050,-5.781,12.505,27.746,-15.718,23.8,-31.8,4.3,142.771,2203.952,3,1,1,0
4,CHN201708010,1,0,1,2,0,0,1,119.29,188.36,83.8,3.224,1.305,7.434,10.215,0.095,1.769,2.484,50,5.596,-8.753,-131.728,-7.278,13.155,26.116,-14.097,23.8,-36.5,4.4,143.956,2462.278,3,1,1,0


In [33]:
px_, py_ = separateData(test_pitches)

testp = clf_.predict(px_)

testa = getAccuracy(testp, py_)

#pd.DataFrame(testp, columns=[''])
test_pitches['classification'] = testp
test_pitches['correct'] = test_pitches.apply(lambda row: (int(row['pitch_event']) == int(row['classification'])), axis=1)

print('test accuracy', testa)

print('py_ array', getCounts(py_))
print('test predictions', getCounts(testp))

#print(test_pitches.head(20))
avg_mid = (test_pitches['sz_top'].mean() + test_pitches['sz_bot'].mean())/2
print("average middle", avg_mid)
avg_ht_range = test_pitches['sz_top'].mean() - test_pitches['sz_bot'].mean()
print("average height range", avg_ht_range)
test_pitches_slim = test_pitches[['retro_game_id', 'px', 'pz', 'sz_top', 'sz_bot', 'pitch_event', 'classification', 'correct']]
print(test_pitches_slim.head())
dfs = dict(tuple(test_pitches_slim.groupby('retro_game_id')))

for item in dfs:
    dfs[item].to_csv('webApp/src/app/test_game_data/test_data_' + item + '.csv', header=True, encoding='utf-8')
    #print("game id", item, dfs[item].shape)

test accuracy 0.591870160810006
py_ array {0: 3004, 2: 1378, 1: 943, 3: 1391}
test predictions {0: 3611, 2: 1532, 3: 1263, 1: 310}
average middle 2.4679111078
average height range 1.84513609291
  retro_game_id     px     pz  sz_top  sz_bot  pitch_event  classification  \
0  CHN201708010 -1.343  0.979   3.501   1.664            0               0   
1  CHN201708010 -0.796  1.656   3.521   1.673            2               0   
2  CHN201708010 -1.084  1.082   3.316   1.397            1               0   
3  CHN201708010 -0.487  2.152   3.317   1.398            3               2   
4  CHN201708010  0.095  1.769   3.224   1.305            3               2   

   correct  
0     True  
1    False  
2    False  
3    False  
4    False  


In [103]:
#print(py)
mdl = logreg()
#print(px.shape)
#print(py['swinging_strike'].shape)
mdl.fit(px[:5000], py[:5000,0])
print('hi')
print(mdl.coef_)

train_predictions = mdl.predict(px[:5000])
test_predictions = mdl.predict(px[5000:])

def peekPredictions(predictions):
    for i in range(50):
        print(i, 'th example, predicted', predictions[i])
        
#peekPredictions(train_predictions)
#peekPredictions(test_predictions)

#print('train_predictions length =', len(train_predictions), 'test_predictions length =', len(test_predictions))
#print('train_predictions shape =', train_predictions.shape, 'test_predictions shape =', test_predictions.shape)
#print('actual train shape =', py['swinging_strike'][:5000].shape, 'actual test shape =', py['swinging_strike'][5000:].shape)
#print('predict.shape =', predictions.shape, 'actual.shape =', py['swinging_strike'].shape)
trainacc = getAccuracy(train_predictions, py[:5000,0])
testacc = getAccuracy(test_predictions, py[5000:,0])
print("train accuracy = ", trainacc)
print("test accuracy = ", testacc)


hi
[[ -1.28208290e-02  -6.82109948e-03   2.55325793e-01   2.16933845e-01
    3.04956398e-02   3.65039709e-01  -7.56428787e-01  -1.62742509e+00
    3.01951378e-01  -8.27995043e-01   1.38733737e+00   1.35674146e-01
    1.12511691e-01   5.63477599e-01  -7.84372675e-03   8.66210514e-02
   -1.08148070e-01   1.20360832e+00  -6.06263761e-03   4.44973399e-02
   -1.60142775e-03   6.33189966e-05]]
0 th example, predicted 0
1 th example, predicted 0
2 th example, predicted 0
3 th example, predicted 0
4 th example, predicted 0
5 th example, predicted 0
6 th example, predicted 0
7 th example, predicted 0
8 th example, predicted 0
9 th example, predicted 0
10 th example, predicted 0
11 th example, predicted 1
12 th example, predicted 0
13 th example, predicted 0
14 th example, predicted 0
15 th example, predicted 1
16 th example, predicted 1
17 th example, predicted 0
18 th example, predicted 0
19 th example, predicted 0
20 th example, predicted 0
21 th example, predicted 1
22 th example, predicted 

In [83]:
#testacc = getAccuracy(test_predictions, py['swinging_strike'][5000:].values)